<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Trodes/Exporting_Trodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import subprocess

In [ ]:
filename = 'C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Test_Emily/AngelfoodCake01/AngelfoodCake01.rec'
os.chdir('C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/')
subprocess.run(["trodesexport.exe", "-raw -rec AngelfoodCake01.rec  -output DT_export"])

CompletedProcess(args=['trodesexport.exe', '-raw -rec C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Test_Emily/AngelfoodCake01/AngelfoodCake01.rec  -output DT_export'], returncode=4294967295)

In [ ]:
trodesexport.exe -raw -rec C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Test_Emily/AngelfoodCake01/AngelfoodCake01.rec -output DT_export

In [ ]:
trodesexport.exe -raw -rec AngelfoodCake01.rec -output DT_export

In [ ]:

!cd C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64
!trodesexport.exe -rec $filename -raw -output DT_BAT.bat

In [ ]:
os.chdir('C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/AngelfoodCake01.raw')
x = readTrodesExtractedDataFile('AngelfoodCake01.raw_group0.dat')

In [ ]:
x

{'description': 'Raw (unfiltered) data for one sorting group',
 'byte_order': 'little endian',
 'original_file': 'AngelfoodCake01.rec',
 'clockrate': '30000',
 'trodes_version': '-1',
 'compile_date': '-1',
 'compile_time': '-1',
 'qt_version': '-1',
 'commit_tag': '-1',
 'controller_firmware': '-1',
 'headstage_firmware': '-1',
 'controller_serialnum': '-1',
 'headstage_serialnum': '-1',
 'autosettle': '-1',
 'smartref': '-1',
 'gyro': '-1',
 'accelerometer': '-1',
 'magnetometer': '-1',
 'time_offset': '0',
 'system_time_at_creation': '-1',
 'timestamp_at_creation': '-1',
 'first_timestamp': '951222',
 'sorting_group': '0',
 'num_channels': '32',
 'voltage_scaling': '0.195',
 'fields': '<voltage 32*int16>',
 'data': array([([ -941, -2416, -2515,  -896,  -800, -1928, -2480, -1313,  -766, -1211,  -555,  -688, -2470, -2975, -2786, -1812,  -282,   -44,   820, -2778, -1690, -2872, -1263, -2487,  -719,   434,   888, -3760, -2526, -1521, -2508, -2445],),
        ([ -884, -2429, -2621,  -871

In [ ]:
#!/usr/bin/python
#Requires numpy to be installed. re and sys are defaults for python
#assumes python 2.7
import numpy as np
import re
from sys import argv
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText


# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)



